Google 에서도 sentencepiece 라는 이름으로 Word Piece Model package 를 공개하였습니다. 설치는 pip install 로 가능합니다. 실습 환경에서의 버전은 `sentencepiece==0.1.8` 입니다.

```
pip install sentencepiece
```

학습 데이터는 빈 칸이 포함되지 않은 문서 집합입니다. 우리의 실습 데이터에는 정규화 과정에서 한문으로만 이뤄진 기사의 텍스트가 지워졌기 때문에 빈 줄이 포함되어 있습니다. 이 빈 줄을 제거하여 `spm_input.txt` 파일을 만듭니다.

In [1]:
import config
import sentencepiece as spm
from navernews_10days import get_news_paths
from soynlp.utils import DoublespaceLineCorpus

path = get_news_paths(date='2016-10-20', tokenize=None)
corpus = DoublespaceLineCorpus(path, iter_sent=True)

input_file = 'spm_input.txt'
with open(input_file, 'w', encoding='utf-8') as f:
    for sent in corpus:
        f.write('{}\n'.format(sent))

soynlp=0.0.491


Sentencepiece 는 C++ 로 구현되어 있으며, subprocess 를 이용하여 파이썬에서 스크립트를 실행시키는 방식입니다. Input 은 반드시 텍스트 파일이어야 합니다. command message 는 input, prefix, vocab_size 를 입력해야 합니다.

IPython notebook 에서는 파이썬 스크립트에서 오류가 날 경우 커널이 그대로 죽습니다. 파이썬 인터프리터에서 직접 작업하면 코드의 오류를 볼 수 있습니다. vocab_size 가 지나치게 작게 설정되면 모델이 학습을 하다 오류를 발생시킵니다. 당황하지 마시고 vocab_size 를 키우면 됩니다. 우리의 실습 예시에서는 vocab_size 1000 으로 설정하면 오류가 발생합니다. 2000 으로 키우면 문제없이 작동합니다.

In [2]:
templates = '--input={} --model_prefix={} --vocab_size={}'

vocab_size = 2000
prefix = '2016-10-20-news'
cmd = templates.format(input_file, prefix, vocab_size)

spm.SentencePieceTrainer.Train(cmd)

학습이 끝나면 `2016-10-20-news` 라는 prefix 를 지니는 .model 파일과 .vocab 파일이 만들어집니다.

In [4]:
ls

2016-10-20-news.model
2016-10-20-news.vocab
config.py
day_3_0_navernews_dataset.ipynb
day_3_1_cohesion_branching_entropy.ipynb
day_3_2_WordPieceModel.ipynb
day_3_3_noun_extraction.ipynb
day_3_4_word_noun_tokenizer_soynlp.ipynb
day_3_sentencepiece_(google_package).ipynb
noun_score_sejong
__pycache__/
README.md
spm_input.txt


학습된 모델을 이용하려면 .model 을 로딩합니다.

In [5]:
sp = spm.SentencePieceProcessor()
sp.Load('{}.model'.format(prefix))

True

문장을 입력하면 subword list 로 출력할 수 있습니다.

In [6]:
sp.EncodeAsPieces('오늘의 연합뉴스 기사입니다')

['▁오늘', '의', '▁연합뉴스', '▁기사', '입니다']

혹은 subword idx list 로 출력할 수도 있습니다.

In [7]:
sp.EncodeAsIds('오늘의 연합뉴스 기사입니다')

[870, 6, 442, 786, 433]

우리가 설정한 2000 개의 vocabulary 가 학습되었습니다.

In [8]:
with open('{}.vocab'.format(prefix), encoding='utf-8') as f:
    vocabs = [doc.strip() for doc in f]

print('num of vocabs = {}'.format(len(vocabs)))

num of vocabs = 2000


Vocabulary 에는 unknown, 문장의 시작, 문장의 끝, 단어의 앞부분 등 네 가지의 special token 이 있으며, 그 외에는 `을`, `이`, `의`, `는` 과 같은 조사들이 뒤따릅니다. `_기자`, `_재배포` 와 같은 도메인에서 자주 이용되는 단어들도 포함되어 있습니다. 

In [9]:
for vocab in vocabs:
    print(vocab)

<unk>	0
<s>	0
</s>	0
▁	-2.31113
을	-4.05915
이	-4.09143
의	-4.2983
는	-4.38238
에	-4.42689
가	-4.55528
를	-4.57877
은	-4.71156
한	-4.88453
도	-4.8983
로	-5.01206
고	-5.04376
지	-5.08105
일	-5.10806
기	-5.15675
다	-5.19795
에서	-5.19852
인	-5.19882
리	-5.30077
▁이	-5.30135
과	-5.3075
사	-5.35807
해	-5.45161
시	-5.45937
자	-5.46572
정	-5.51052
으로	-5.53279
▁수	-5.54834
했다	-5.61828
원	-5.62005
▁전	-5.65052
대	-5.6526
▁있다	-5.66691
스	-5.67296
장	-5.72082
와	-5.72919
수	-5.73534
▁20	-5.76771
어	-5.77456
주	-5.77855
상	-5.78118
하고	-5.83977
▁고	-5.84274
▁등	-5.85273
하는	-5.86349
진	-5.9251
아	-5.94291
부	-5.95611
제	-5.9649
라	-5.96523
구	-5.98013
▁기자	-5.98875
서	-6.00749
년	-6.00979
성	-6.03283
할	-6.04627
만	-6.05573
동	-6.06602
▁김	-6.09988
전	-6.10718
나	-6.13431
보	-6.14256
▁대	-6.15682
화	-6.16392
1	-6.17084
▁및	-6.17976
▁1	-6.20694
▁무단	-6.22084
▁서울	-6.22214
회	-6.24648
면	-6.26359
소	-6.26558
치	-6.26763
계	-6.26962
조	-6.27293
▁3	-6.27301
트	-6.28715
적	-6.30156
▁금지	-6.30178
▁재배포	-6.32423
월	-6.3337
▁지	-6.33896
▁그	-6.34329
세	-6.34353
▁중	-6.35338
드	-6.35

숲	-9.90815
돋	-9.91374
엽	-9.93263
옷	-9.94221
숨	-9.94444
넥	-9.94607
쇄	-9.96361
밍	-9.96558
넷	-9.96754
쌀	-9.97548
셈	-9.98548
찌	-9.99965
닐	-10.0058
뚜	-10.0161
톱	-10.0161
릿	-10.0223
셉	-10.0244
닌	-10.0293
쁜	-10.0328
섯	-10.0671
꾼	-10.0693
닷	-10.0737
씬	-10.0737
놨	-10.0781
뒷	-10.0803
땅	-10.0803
쓸	-10.0803
줘	-10.0847
샵	-10.0914
틱	-10.0914
빙	-10.0936
훼	-10.0981
딤	-10.0981
뽑	-10.1163
묘	-10.1186
윈	-10.1325
젠	-10.1371
섹	-10.1371
옆	-10.1418
멜	-10.1442
잉	-10.1442
덜	-10.1465
랍	-10.1584
롱	-10.1632
앨	-10.168
듬	-10.168
쿄	-10.168
렛	-10.1728
꿀	-10.1777
쁘	-10.1777
컷	-10.1801
릉	-10.185
껴	-10.185
눔	-10.1875
렵	-10.1899
똑	-10.1924
뭐	-10.1974
템	-10.1998
쏘	-10.2024
롤	-10.2099
빚	-10.2099
흑	-10.2099
칩	-10.2149
룩	-10.2175
굉	-10.2276
깜	-10.2302
덴	-10.2379
킷	-10.2379
빵	-10.2405
팝	-10.2483
촬	-10.2534
싼	-10.2536
껏	-10.2562
잖	-10.2668
므	-10.2695
젤	-10.2748
흉	-10.2775
댄	-10.2802
밸	-10.2856
쥐	-10.2993
뿌	-10.3021
첩	-10.3048
밭	-10.3104
썬	-10.3159
옛	-10.3159
콕	-10.3159
눠	-10.3329
옵	-10.3357
팩	-10.3386
쌍	-10.3443
믹	-10.3559
앙	-1